In [1]:
import itertools

import pandas as pd
import numpy as np
import pynetlogo
from tqdm import tqdm

/var/folders/51/wgmtx7ys3731mhrjy2sngf1r0000gp/T/ipykernel_61812/3467248944.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Connect to NetLogo

In [2]:
netlogo = pynetlogo.NetLogoLink(gui=False)
netlogo.load_model('Altruism_RMAI.nlogo')

### Define Experiment Parameters

In [3]:
# length of each experiment
TICKS = 10

# how many to run each experiment
N_RUNS = 1

experiment_parameters = {
    'occupation-prob': [0.1, 0.5, 0.9],
    'occupation-radius': [10, 20, 40],
    'occupation-diffusion': [0.2, 0.6, 1],
    'prob-gain-resource': [0.1, 0.5, 0.9],
    'agent-move-cost': [1, 40, 80],
    'stride-length': [0.1, 0.7, 0.8],
    'resource-energy': [20, 60, 99],
    'altruism-resource-threshold': [1, 80, 99],
    'reproduction-threshold': [1, 80, 99],
    'reproduction-cost': [1, 60, 99]
}

### Make list of all parameter combinations

In [5]:
keys, values = zip(*experiment_parameters.items())
parameter_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
parameter_combinations = [param for param in parameter_combinations for _ in range(0, N_RUNS)]
len(parameter_combinations)

59049

### Run experiments

In [6]:
results = []

for params in tqdm(parameter_combinations):
    netlogo.command('setup')

    for param, value in params.items():
        netlogo.command(f'set {param} {value}')
    
    netlogo.command(f'repeat {TICKS} [ go ]')  
    
    # Collect results
    count_altruism_agents = netlogo.report('count altruism-agents')
    count_greedy_agents = netlogo.report('count greedy-agents')
    n_ticks = netlogo.report('ticks')

    results.append({**params, 'Altruism Agents': count_altruism_agents, 'Greedy Agents': count_greedy_agents, 'Number of ticks': n_ticks})
    

  0%|                                     | 11/59049 [00:06<10:23:25,  1.58it/s]


KeyboardInterrupt: 

In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df